In [ ]:
# Rynkeby Thor
# Installer pakker
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import xml.etree.ElementTree as ET
from google.colab import drive
from torchvision import transforms

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

drive.mount('/content/gdrive', force_remount=True)
os.chdir('/content/gdrive/My Drive/Projekt20TeamRynkeby')

Mounted at /content/gdrive


In [ ]:
### Rynkeby 
# Class
import os, psutil
process = psutil.Process(os.getpid())


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        n = 1500
        start = 1
        self.imgs = list(sorted(os.listdir("Filtered_frames2")))#[start:start+n]#
        self.allxml = sorted(os.listdir('framesXML2'))#[start:start+n]#
        self.allFilterFrames = sorted(os.listdir('Filtered_frames2'))#[start:start+n]#

    def __getitem__(self, idx):

        #framenumber = f"{idx:06}"
        xml = self.allxml[idx]
        
        #tree = ET.parse('project_20_data/video1/framesXML/frame_{}.xml'.format(framenumber))
        tree = ET.parse('framesXML2/{}'.format(xml))      
        root = tree.getroot()

        labels = [0] * 10
        imgString = self.allFilterFrames[idx]
        #print(imgString)
        objects = 0
        var=0
        # print all the objects
        for obj in root.iter('object'):
          name = obj.find('name').text

          if name == 'beer':
            var = 1
          elif name == 'cola':
            var = 2
          #else:
          #  var = 420
          #print('var: {}'.format(var))
          # print('idx: {}'.format(idx))
          # print('imgString: {}\n'.format(imgString))    
          process = psutil.Process(os.getpid())
          #print('in bytes: {}\n'.format(process.memory_info().rss))   

          labels[objects] = var
          objects = objects + 1
          
          #labels.append(var)
          #print((labels))
        boxes = []
        area = []
        #print(labels)
        if all(v == 0 for v in labels):
          ghjksd=0
          #print('No objects in image')
        else:
        #if labels: #it calculates area if there are Labels
          # Find and save all coordinates of box for each object
          for features in root.iter('bndbox'):

            xmin = float(features.find('xmin').text)
            xmax = float(features.find('xmax').text)
            ymin = float(features.find('ymin').text)
            ymax = float(features.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])

          boxes = torch.as_tensor(boxes, dtype=torch.float32)
          area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
          area = torch.as_tensor(area,dtype=torch.float32)
          
        # Test if no boxes
        if objects == 0:
          boxes = torch.empty(0,4)
          area = torch.tensor([0])

        # Make sure the boxes is converted even if there is no objects
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = list(filter((0).__ne__, labels))
        labels = torch.as_tensor(labels, dtype=torch.int64)

        img_path = 'Filtered_frames2/{}'.format(imgString)
        img = Image.open(img_path).convert('RGB')
        #display(img)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.as_tensor(idx, dtype=torch.int64)
        target["area"] = area
        target["iscrowd"] = torch.ones(objects, dtype=torch.uint8)

        if self.transforms is not None:      
            img, target = self.transforms(img, target)
        return img, target

    def __len__(self):
        return len(self.imgs)

That's all for the dataset. Let's see how the outputs are structured for this dataset

In [ ]:
#dataset = PennFudanDataset('')

#Undersøg data
# step = int(np.rint(len(dataset)*0.05))
# for i in range(0,len(dataset)-1,step):
#    print(i)
#    display(dataset[i][0])

So we can see that by default, the dataset returns a `PIL.Image` and a dictionary
containing several fields, including `boxes`, `labels` and `masks`.

That's it, this will make model be ready to be trained and evaluated on our custom dataset.

## Training and evaluation functions

In `references/detection/,` we have a number of helper functions to simplify training and evaluating detection models.
Here, we will use `references/detection/engine.py`, `references/detection/utils.py` and `references/detection/transforms.py`.

Let's copy those files (and their dependencies) in here so that they are available in the notebook

In [ ]:
'''
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2


cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../
'''

'\n%%shell\n\n# Download TorchVision repo to use some files from\n# references/detection\ngit clone https://github.com/pytorch/vision.git\ncd vision\ngit checkout v0.8.2\n\n\ncp references/detection/utils.py ../\ncp references/detection/transforms.py ../\ncp references/detection/coco_eval.py ../\ncp references/detection/engine.py ../\ncp references/detection/coco_utils.py ../\n'



Let's write some helper functions for data augmentation / transformation, which leverages the functions in `refereces/detection` that we have just copied:


In [ ]:
# Genstart kørsels tid, hvis den ikke virker
from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
    transforms = []
    # converts the image from a PIL image to a PyTorch Tensor
    transforms.append(T.ToTensor())
    
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

#### Testing forward() method 

Before iterating over the dataset, it’s good to see what the model expects during training and inference time on sample data.


#### Note that we do not need to add a mean/std normalization nor image rescaling in the data transforms, as those are handled internally by the Mask R-CNN model.

### Putting everything together

We now have the dataset class, the models and the data transforms. Let's instantiate them

In [ ]:
# use our dataset and defined transformations
dataset = PennFudanDataset('',get_transform(train=True))
dataset_test = PennFudanDataset('',get_transform(train=False))

# split the dataset in train and test set
# torch.manual_seed(1)

indices= torch.arange(0,0+len(dataset)).tolist() # Denne linje er ny

dataset = torch.utils.data.Subset(dataset, indices[:-300])

dataset_test = torch.utils.data.Subset(dataset_test, indices)#[-200:]

n = 4
m = 4
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=n, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=m, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

TrainTal = len(dataset)/n
TestTal = len(dataset_test)/m
print('TrainTal er :{}\nTestTal  er :{}'.format(TrainTal,TestTal))

TrainTal er :300.0
TestTal  er :375.0


Now let's instantiate the model and the optimizer

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
# This dataset contains 3 classes: Beer (1), Coke (2) and background (0)
num_classes = 3

# Get network
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# move model to the right device
model.to(device)

in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)
print(device)

cuda


And now let's train the model for 10 epochs, evaluating at the end of every epoch.

In [ ]:
# # let's train it for some epochs
# from torch.optim.lr_scheduler import StepLR
# num_epochs = 3 # ctr shift 7

# for epoch in range(num_epochs):
#     # train for one epoch, printing every 10 iterations
#     train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=25)
#     # update the learning rate
#     lr_scheduler.step()
#     # evaluate on the test dataset
#     evaluate(model, data_loader_test, device=device)

In [ ]:
model = torch.load('/content/gdrive/My Drive/Projekt20TeamRynkeby/model:LR=0.005,MOM=0.9, WD=0.0005,SS=3,G=0.1,EPOCHS=3.pt')

Now that training has finished, let's have a look at what it actually predicts in a test image

In [ ]:
# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
class centroidTyuracker():
	def __init__(self, maxDisappeared=5):
		# initialize the next unique object ID along with two ordered
		# dictionaries used to keep track of mapping a given object
		# ID to its centroid and number of consecutive frames it has
		# been marked as "disappeared", respectively
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.disappeared = OrderedDict()
		self.ClassO = 1#np.zeros(100)
		# store the number of maximum consecutive frames a given
		# object is allowed to be marked as "disappeared" until we
		# need to deregister the object from tracking
		self.maxDisappeared = maxDisappeared
		print('__init__')
	

         
	def register(self, centroid):
		#print('Register')
		# when registering an object we use the next available object
		# ID to store the centroid
		self.objects[self.nextObjectID] = centroid
		self.disappeared[self.nextObjectID] = 0
	
		self.nextObjectID += 1

	def deregister(self, objectID):
		#print('In deregister')
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
		del self.objects[objectID]
		del self.disappeared[objectID]

	def update(self, rects): #,classI
		# check to see if the list of input bounding box rectangles
		# is empty
		if len(rects) == 0:
			# loop over any existing tracked objects and mark them
			# as disappeared
			for objectID in list(self.disappeared.keys()):
				self.disappeared[objectID] += 1
				# if we have reached a maximum number of consecutive
				# frames where a given object has been marked as
				# missing, deregister it
				if self.disappeared[objectID] > self.maxDisappeared:
					#print('rect =0')
					self.deregister(objectID)
			# return early as there are no centroids or tracking info
			# to update
			return self.objects
		# initialize an array of input centroids for the current frame
		inputCentroids = np.zeros((len(rects), 2), dtype="int")
		# loop over the bounding box rectangles
		for (i, (startX, startY, endX, endY)) in enumerate(rects):
			# use the bounding box coordinates to derive the centroid
			cX = int((startX + endX) / 2.0)
			cY = int((startY + endY) / 2.0)
			inputCentroids[i] = (cX, cY)
		# if we are currently not tracking any objects take the input
		# centroids and register each of them

		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i])
		# otherwise, are are currently tracking objects so we need to
		# try to match the input centroids to existing object
		# centroids
		else:
			# grab the set of object IDs and corresponding centroids
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())
			# compute the distance between each pair of object
			# centroids and input centroids, respectively -- our
			# goal will be to match an input centroid to an existing
			# object centroid
			D = dist.cdist(np.array(objectCentroids), inputCentroids)
			# in order to perform this matching we must (1) find the
			# smallest value in each row and then (2) sort the row
			# indexes based on their minimum values so that the row
			# with the smallest value is at the *front* of the index
			# list
			#rows = D.min(axis=1).argsort()
			# next, we perform a similar process on the columns by
			# finding the smallest value in each column and then
			# sorting using the previously computed row index list
			#cols = D.argmin(axis=1)[rows]
			cols = D.min(axis=0).argsort()
			rows = D.argmin(axis=0)[cols]
			# in order to determine if we need to update, register,
			# or deregister an object we need to keep track of which
			# of the rows and column indexes we have already examined
			usedRows = set()
			usedCols = set()
			# loop over the combination of the (row, column) index
			# tuples
			for (row, col) in zip(rows, cols):
				# if we have already examined either the row or
				# column value before, ignore it
				# val
				if row in usedRows or col in usedCols:
					continue
				# otherwise, grab the object ID for the current row,
				# set its new centroid, and reset the disappeared
				# counter
				objectID = objectIDs[row]
				self.objects[objectID] = inputCentroids[col]
				self.disappeared[objectID] = 0
				# indicate that we have examined each of the row and
				# column indexes, respectively
				usedRows.add(row)
				usedCols.add(col)
			# compute both the row and column index we have NOT yet
			# examined
			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)
			# in the event that the number of object centroids is
			# equal or greater than the number of input centroids
			# we need to check and see if some of these objects have
			# potentially disappeared
			if D.shape[0] >= D.shape[1]:
				# loop over the unused row indexes
				for row in unusedRows:
					# grab the object ID for the corresponding row
					# index and increment the disappeared counter
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1
					# check to see if the number of consecutive
					# frames the object has been marked "disappeared"
					# for warrants deregistering the object
					if self.disappeared[objectID] > self.maxDisappeared:
						#print('rect s0')
						self.deregister(objectID)
			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
			else:
				for col in unusedCols:
					self.register(inputCentroids[col])
		# return the set of trackable objects
		self.ClassO=1

		return self.objects


In [ ]:
# import the necessary packages
from collections import namedtuple
import numpy as np
import cv2
# define the `Detection` object
# (startX, startY, endX, endY)
def bb_intersection_over_union(boxA, boxB):

	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	#print(yB)
	submerging = 0
	if (boxA[0] > boxB[0]) and (boxA[1] > boxB[1]) and (boxA[2] < boxB[2]) and (boxA[3] < boxB[3]):
		submerging = 1
	if (boxA[0] < boxB[0]) and (boxA[1] < boxB[1]) and (boxA[2] > boxB[2]) and (boxA[3] > boxB[3]):
		submerging = 1

	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)

	# return the intersection over union value
	return iou, submerging


In [ ]:
def UniqueBox(predictions):
  #Determining unike boxes
  using = np.ones(len(predictions))
  value=0
  for idx1 in range(len(predictions)):

    for idx2 in range(len(predictions)):

      uiden = bb_intersection_over_union(predictions[idx1], predictions[idx2])

      if ((uiden[0]>0.3) or (uiden[1] == 1)) and (idx1 < idx2):
        using[idx2] = value
        #value = value+1
  for i in range(len(predictions)):
    if (using[i] == 1):
      using[i] = i+1
  
  return using

In [ ]:
def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a

In [ ]:
len(dataset_test)

1500

In [ ]:
from imutils.video import VideoStream
import argparse
import imutils
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches


ct = centroidTyuracker()

for PicNum in range (1200):
  # pick one image from the test set
  test_image = PicNum+776
  img, _ = dataset_test[test_image]
  # put the model in evaluation mode
  model.eval()
  with torch.no_grad():
      prediction = model([img.to(device)])

  # Make threshold for scores
  scores = (prediction[0]['scores'])
  threshhold = (scores > 0.95).nonzero(as_tuple=True)[0]# Counter. Hver gang score > T, tilføjes nyt tal til rækken [0,1,...N]

  # Create figure and axes
  fig, ax = plt.subplots()

  # Display the image
  Image.fromarray(prediction[0]['boxes'].mul(255).byte().cpu().numpy())
  tensor_image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
  ax.imshow(tensor_image)

  rects = []
  Unique = UniqueBox(prediction[0]['boxes']) #The unique idendified boxes
  NrOfObjects=len((np.where(Unique != 0))[0])# Amount of unique objects
  #print('Amount of Unique objects: {}'.format(NrOfObjects))
  if (len(Unique)!=0):

    # Gennemgår alle boxes med score over threshhold
    edgecolor=[]
    CanClass = []
    for i in range(len(threshhold)):
      
      if (Unique[i]!=0): #only use the unique boxes:
        # Box
        B = prediction[0]['boxes'][threshhold[i]].cpu().squeeze().numpy()
        
        # Hvis det er den første box:
        if i == 0:
          rects=np.append([B],[B], axis=0)
        
        else:
          rects=np.append(rects,[B], axis=0)
        
        # Sørger for at boxen for i == 0 ikke optræder 2 gange
        rects=rects[0:i+1]
        rects = totuple(rects)
        # Create a Rectangle patch (start coordinate), width, hight


        # Add label color
        if prediction[0]['labels'][threshhold[i]].cpu().squeeze().numpy() == 1:
          edgecolor.append('lime')
          CanClass.append(1)

        if prediction[0]['labels'][threshhold[i]].cpu().squeeze().numpy() == 2:
          edgecolor.append('red')
          CanClass.append(2)

        box = patches.Rectangle((B[0], B[1]), B[2]-B[0], B[3]-B[1], linewidth=1, edgecolor=edgecolor[i], facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(box) #makebox
        
        # Add score to the boxes
        score = round(prediction[0]['scores'][threshhold[i]].cpu().squeeze().numpy()*100,2)

        rx, ry = box.get_xy()
        cx = rx + box.get_width()
        cy = ry + box.get_height()
        ax.annotate(score, (10+cx, cy), color=edgecolor[i], weight='bold', fontsize=10) #Score%


    objects = ct.update(rects)






    #if objects is not None and objects != OrderedDict() and type(objects) is not OrderedDict:
    centroid = []
    objectID =[]
    for (objID, cend) in objects.items():
      centroid.append(cend)
      objectID.append(objID)





    match=[]
    diff=0
    for j in range(len(rects)):
      for u in range(len(objects)):

        y=0
        factorx=(rects[0][2]-rects[0][0])*0.2
        factory=(rects[0][3]-rects[0][1])*0.2
        re=[rects[j][0]+factorx,rects[j][1]+factory,rects[j][2]-factorx,rects[j][3]-factory]

        box=[centroid[u][0],centroid[u][1],centroid[u][0],centroid[u][1]]
        y=bb_intersection_over_union(box, re)[1]
        if (y==1):
          match.append(u)

    for i in range(len(match)):

        #Centroid
      if (CanClass[i]==1):
        obj = 'Beer'
        color = 'lime'
      if (CanClass[i]==2):
        obj = 'Coke'
        color = 'red'

      plt.scatter(centroid[match[i]][0], centroid[match[i]][1],color=color)
      ax.annotate('{} {}'.format(obj,objectID[match[i]]+1), (centroid[match[i]][0]+50, centroid[match[i]][1]), color=color, weight='bold', fontsize=10) #Object og Nr

    plt.show()
  # Show images with boxes


In [ ]:
objects.items()


odict_items([(3, array([169,  57])), (6, array([239, 335]))])

In [ ]:
centroid[match[i]]


array([411, 326])

In [ ]:
objectIDD=[]
for (objectID, cend) in objects.items():
  objectIDD.append((objectID))
print(objectIDD)

[0, 1, 2]


In [ ]:
objects is not None and objects != OrderedDict() and type(objects) is not OrderedDict

False

In [ ]:
for (objectID, centroid) in objects.items():
  print(centroid)

[227 317]
[334 127]
[411 326]


In [ ]:
print('rects is:            {}'.format(rects))
print('Unique is:           {}'.format(Unique))
print('NrObje is:           {}'.format(NrOfObjects))
print('Box is:              {}'.format(B))
print('Len(T) is:           {}'.format(len(threshhold)))

#print('Predicted label is:  {}'.format(prediction[0]['labels'][threshhold[i]].cpu().squeeze().numpy()))


print('Predicted Boxes are: {}'.format(prediction[0]['boxes'][threshhold[i]].cpu().squeeze().numpy()))
print('Type of boxes are  : {}'.format(type(prediction[0]['boxes'][threshhold[i]].cpu().squeeze().numpy())))
print('Len of unique is:    {}'.format(len(Unique)))
# print(prediction[0])
# print(prediction)
print('All objects observed:\n{}'.format(objects))
print('objectID             {}'.format(objectID))

rects is:            ((190.98041, 268.44498, 264.6262, 365.7258), (288.77316, 87.79468, 380.10934, 167.98907), (370.46277, 285.23688, 453.4522, 367.9566))
Unique is:           [1. 2. 3. 0.]
NrObje is:           3
Box is:              [370.46277 285.23688 453.4522  367.9566 ]
Len(T) is:           3
Predicted Boxes are: [370.46277 285.23688 453.4522  367.9566 ]
Type of boxes are  : <class 'numpy.ndarray'>
Len of unique is:    4
All objects observed:
OrderedDict([(0, array([227, 317])), (1, array([334, 127])), (2, array([411, 326]))])
objectID             2
